In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from glob import glob 
from os.path import join as opj
import os

def summarize(df):
    # remove same round correlations
    df = df.loc[df['round_x'] != df['round_y']]
    #df = df.loc[df['trial_x'] != df['trial_y']]
    
    # define trial type
    conditions = [
    (df['pair_x'] != df['pair_y']),
    (df['destination_x'] != df['destination_y']),
    (df['destination_x'] == df['destination_y'])
    ]
    values = ['across', 'within', 'same']
    df['type'] = np.select(conditions, values)
    
    # define valid type
    conditions = [
    (df['valid_x'] != df['valid_y']),
    (df['valid_x'] == True),
    (df['valid_x'] == False)
        ]
    values = ['valid-invalid', 'valid-valid', 'invalid-invalid']
    df['valid'] = np.select(conditions, values)
    
    return df
    
def group(df, li):
    # mean correlations
    df = df.groupby(li)['cor'].mean().reset_index()
    df['within_trial_TR'] = df['within_trial_TR_x']
    df = df.drop(columns=['within_trial_TR_x'])
    
    return df
    
rois_dict = {
    'ppa_mni_2_epi_thre_0.5_masked':'ppa',
    'ca23dg-body_thre_0.5_masked':'ca23dg-body'
    }
    #'ca1-body_thre_0.5_masked':'ca1-body',
    
    #'evc_2_epi_thre_0.5_masked':'evc'
    #'ca23dg_thre_0.5_masked':'ca23dg',
    #'ca1_thre_0.5_masked':'ca1',
#}

output_dir = "/home/wanjiag/projects/MONSTERA/derivatives/csv_files/python/"
processed_subs = os.listdir(output_dir)

summary_dir = "/home/wanjiag/projects/MONSTERA/derivatives/csv_files/python_summary/"
processed_subs = os.listdir(output_dir)

file_dir = []


In [2]:
def segment(x):
    if x <= 6:
        return 'same'
    elif x <= 12:
        return 'early-similar'
    elif x <= 18:
        return 'late-similar'
    elif x <= 24:
        return 'different'
    else:
        return None


In [ ]:
summary = pd.DataFrame()
for k,roi in rois_dict.items():
    print(roi)
    file_dir = glob(opj(output_dir, '*', 'sub-MONSTERA*_rolling3_{}.csv'.format(roi)))
    print(len(file_dir))
    
    df = pd.concat((pd.read_csv(f) for f in file_dir), ignore_index=True)
    
    output_df = summarize(df)
    
    output_df['segment'] = df['within_trial_TR_x'].apply(segment)
    smaller_output_df = output_df.drop(columns=['x','y','sub_y','TR_x','segment_x','TR_y','segment_y','within_trial_TR_y'])
    
    smaller_output_df_swap = smaller_output_df.copy()
    smaller_output_df_swap = smaller_output_df_swap.rename(columns={"round_x": "round_y", "round_y": "round_x",
                                       "trial_x": "trial_y", "trial_y": "trial_x",
                                       "pair_x": "pair_y", "pair_y": "pair_x",
                                       "destination_x": "destination_y", "destination_y": "destination_x",
                                       "valid_x": "valid_y", "valid_y": "valid_x"})
    
    together = pd.concat([smaller_output_df, smaller_output_df_swap])
    li = ['sub_x','type','valid', 'round_x','trial_x','segment']
    roi_summary = together.groupby(li)['cor'].mean().reset_index()
    roi_summary = roi_summary.pivot(index=['sub_x','valid','round_x','trial_x','segment'], columns='type', values='cor').reset_index()
    roi_summary['roi'] = roi
    
    summary = pd.concat([summary, roi_summary])

ppa
40
ca23dg-body
40


In [ ]:
summary['within_across'] = summary['within'] - summary['across']

In [14]:
summary_valid = summary.loc[summary['valid'] == 'valid-valid']

In [15]:
summary_valid = summary_valid.drop(columns = ['across','within','same','valid']).pivot(index=['sub_x','round_x','trial_x'],columns = ['segment','roi'], values = 'within_across').reset_index()

In [ ]:
tmp = summary_valid.drop(columns = ['round_x', 'trial_x']).groupby(['sub_x']).corr() 

In [18]:
tmp= tmp.reset_index()

In [19]:
tmp.columns = ["_".join(pair) for pair in tmp.columns]

In [20]:
tmp.columns 

Index(['sub_x_', 'segment_', 'roi_', 'different_ca23dg-body',
       'early-similar_ca23dg-body', 'late-similar_ca23dg-body',
       'same_ca23dg-body'],
      dtype='object')

In [ ]:
tmp.columns = ['sub', 'segment.x', 'roi.x', 'different_ca23dg-body',
       'early-similar_ca23dg-body', 'late-similar_ca23dg-body',
       'same_ca23dg-body', 'different_ppa', 'early-similar_ppa',
       'late-similar_ppa', 'same_ppa', 'different_ca1-body',
       'early-similar_ca1-body', 'late-similar_ca1-body', 'same_ca1-body',
       'different_evc', 'early-similar_evc', 'late-similar_evc', 'same_evc']

In [ ]:
melt_tmp = tmp.melt(id_vars=['sub', 'segment.x', 'roi.x'], var_name="type")

In [ ]:
melt_tmp[['segment.y', 'roi.y']] = melt_tmp['type'].str.split('_', expand=True)

In [ ]:
melt_tmp = melt_tmp.drop(columns = ['type'])

In [ ]:
melt_tmp_summary = melt_tmp.groupby(['segment.x','roi.x','segment.y','roi.y']).mean().reset_index()

In [ ]:
melt_tmp_summary.loc[(melt_tmp_summary['segment.x'] =='early-similar') & 
                     (melt_tmp_summary['roi.x'] =='ca23dg-body') &
                     (melt_tmp_summary['segment.y'] =='late-similar')]

In [ ]:
melt_tmp_summary.loc[(melt_tmp_summary['segment.x'] =='early-similar') & 
                     (melt_tmp_summary['roi.x'] =='ppa') &
                     (melt_tmp_summary['segment.y'] =='late-similar')]

In [179]:
import scipy.stats as stats 
t_statistic, p_value = stats.ttest_1samp(
    a=melt_tmp.loc[(melt_tmp['segment.x'] =='same') & 
             (melt_tmp['segment.y'] =='different') &
             (melt_tmp['roi.x'] =='ca23dg-body') &
             (melt_tmp['roi.y'] =='ppa')]['value'], popmean=0) 

In [180]:
print(t_statistic , p_value)

0.34841137690819934 0.7294050852997696
